Futuros pasos:

* Mantener user_id como string ( es un identificador alfanumérico).
* Convertir has_loan a booleana (Sí/No --> True/False).
* Pasar columna record_date a datetime para análisis temporal.

# 1.-Librerías

In [2]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd #manejo de estructura de datos tabulares

# Visualización de datos
# -----------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns


# importamos los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')

# importamos configuracion de pandas
# -----------------------------------------------------------------------
pd.options.display.max_columns = None

# 2.- Cargar datos

In [3]:
# Cargamos los datos desde la carpeta data/raw
df = pd.read_csv("../data/raw/synthetic_personal_finance_dataset.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32424 entries, 0 to 32423
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   user_id                  32424 non-null  object 
 1   age                      32424 non-null  int64  
 2   gender                   32424 non-null  object 
 3   education_level          32424 non-null  object 
 4   employment_status        32424 non-null  object 
 5   job_title                32424 non-null  object 
 6   monthly_income_usd       32424 non-null  float64
 7   monthly_expenses_usd     32424 non-null  float64
 8   savings_usd              32424 non-null  float64
 9   has_loan                 32424 non-null  object 
 10  loan_type                12995 non-null  object 
 11  loan_amount_usd          32424 non-null  float64
 12  loan_term_months         32424 non-null  int64  
 13  monthly_emi_usd          32424 non-null  float64
 14  loan_interest_rate_pct